In [159]:
import logging
logging.root.handlers = []  # Jupyter messes up logging so needs a reset
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)
from smart_open import smart_open
import pandas as pd
import numpy as np
from numpy import random
import gensim
import nltk
from sklearn.cross_validation import train_test_split
from sklearn import linear_model
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics import accuracy_score, confusion_matrix
import matplotlib.pyplot as plt
from gensim.models import Word2Vec
from sklearn.neighbors import KNeighborsClassifier
from sklearn import linear_model
from nltk.corpus import stopwords
import gensim
import warnings
warnings.filterwarnings(action='ignore', category=UserWarning, module='gensim')
from gensim.models import KeyedVectors
from sklearn.multiclass import OneVsRestClassifier
from sklearn.pipeline import Pipeline
from sklearn.svm import LinearSVC
from sklearn.svm import SVC
from sklearn.model_selection import cross_val_score
from sklearn.svm import SVC
from sklearn.svm import LinearSVC
from sklearn import svm
from sklearn.metrics import classification_report
from sklearn.metrics import f1_score

In [136]:
df = pd.read_csv('data/Shuffle_Data.csv')
df = df.dropna()
df['content'].apply(lambda x: len(x.split(' '))).sum()

434630

In [137]:
X, y = df['content'], df['tag']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state = 42)

In [138]:
%%time 
wv = KeyedVectors.load_word2vec_format('GoogleNews-vectors-negative300.bin', binary=True,limit=1500000)
wv.init_sims(replace=True)

2018-04-18 06:13:28,304 : INFO : loading projection weights from GoogleNews-vectors-negative300.bin
2018-04-18 06:13:55,192 : INFO : loaded (1500000, 300) matrix from GoogleNews-vectors-negative300.bin
2018-04-18 06:13:57,266 : INFO : precomputing L2-norms of word weight vectors


Wall time: 41.2 s


In [139]:
word1 = "treachery"
wv.most_similar (positive=word1)

[('betrayal', 0.6599762439727783),
 ('duplicity', 0.6232621669769287),
 ('perfidy', 0.5984146595001221),
 ('deceit', 0.586785614490509),
 ('cowardice', 0.5809214115142822),
 ('betrayals', 0.5602282285690308),
 ('barbarity', 0.5583377480506897),
 ('unscrupulousness', 0.5564166903495789),
 ('wickedness', 0.5546039342880249),
 ('skulduggery', 0.5511748790740967)]

In [140]:
def word_averaging(wv, words):
    all_words, mean = set(), []
    
    for word in words:
        if isinstance(word, np.ndarray):
            mean.append(word)
        elif word in wv.vocab:
            mean.append(wv.vectors_norm[wv.vocab[word].index])
            all_words.add(wv.vocab[word].index)

    if not mean:
        logging.warning("cannot compute similarity with no input %s", words)
        # FIXME: remove these examples in pre-processing
        return np.zeros(wv.layer1_size,)

    mean = gensim.matutils.unitvec(np.array(mean).mean(axis=0)).astype(np.float32)
    return mean

def  word_averaging_list(wv, text_list):
    return np.vstack([word_averaging(wv, review) for review in text_list ])

In [141]:
def w2v_tokenize_text(text):
    tokens = []
    for sent in nltk.sent_tokenize(text, language='english'):
        for word in nltk.word_tokenize(sent, language='english'):
            if len(word) < 2:
                continue
            tokens.append(word)
    return tokens

In [142]:
test_tokenized = X_test.to_frame().apply(lambda r: w2v_tokenize_text(r['content']), axis=1).values
train_tokenized = X_train.to_frame().apply(lambda r: w2v_tokenize_text(r['content']), axis=1).values

In [143]:
X_train_word_average = word_averaging_list(wv,train_tokenized)
X_test_word_average = word_averaging_list(wv,test_tokenized)

In [144]:
clf = OneVsRestClassifier(LinearSVC(C=15)) # C=1.0, max_iter=1000

score = cross_val_score(clf, X_train_word_average,  y_train.to_frame()['tag'], cv=5)

# print((train_data_features))
# print((y_train))
# print (score)
cv = np.mean(score)
print ("Accuracy using SVM with 5 cross validation : {}%".format(round(cv*100,2)))

Accuracy using SVM with 5 cross validation : 71.35%


In [145]:
clf.fit(X_train_word_average, y_train.to_frame()['tag'])

OneVsRestClassifier(estimator=LinearSVC(C=15, class_weight=None, dual=True, fit_intercept=True,
     intercept_scaling=1, loss='squared_hinge', max_iter=1000,
     multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
     verbose=0),
          n_jobs=1)

## testing eval

In [146]:
pred = clf.predict(X_test_word_average)
accuracy = clf.score(X_test_word_average,y_test.to_frame().tag)
print (accuracy)

0.803921568627451


In [147]:
X_testArray = X_test.values
y_testArray = y_test.values

In [148]:
# row : actual :: column : predicted 
cf = confusion_matrix(y_testArray,pred)
print (cf)

[[12  5  0]
 [ 2 18  1]
 [ 1  1 11]]


In [150]:
#  total classes
from __future__ import division

In [163]:
total = 3
classes = {0:'Homicide', 1:'Parricide', 2:'Murder'}
category = 0

In [164]:
TrueP = []
FalseN = []
FalseP = []
TrueN = []

pre = 0
rec = 0

In [165]:
for category in range(total):
    TP = FN = FP = TN = 0 
    for i in range(total):
        for j in range(total):
            if i==category and j==category:
                TP += cf[i][j]
            elif i==category and j!=category:
                FN += cf[i][j]
            elif i!=category and j==category:
                FP += cf[i][j]
            else:
                TN += cf[i][j]
    print (classes.get(category))
    print (TP, FN, FP, TN)
    print ("Precision: " , TP/(TP+FP))
    print ("Recall: " , TP/(TP+FN))
    print ("\n")
    TrueP.append(TP)
    FalseN.append(FN)
    FalseP.append(FP)
    TrueN.append(TN)
    pre += (TP/(TP+FP))
    rec += TP/(TP+FN)

print ("pre: ", pre/3)
print ("rec:", rec/3)
    
# Average confusion matrix for all classes
TP = np.sum(TrueP)
FN = np.sum(FalseN)
FP = np.sum(FalseP)
TN = np.sum(TrueN)  

# print  ("",  TP, FN, FP, TN, ((TP+TN)/(TP+TN+FP+FN)))

Homicide
12 5 3 31
Precision:  0.8
Recall:  0.7058823529411765


Parricide
18 3 6 24
Precision:  0.75
Recall:  0.8571428571428571


Murder
11 2 1 37
Precision:  0.9166666666666666
Recall:  0.8461538461538461


pre:  0.8222222222222223
rec: 0.8030596854126265


In [166]:
# calculate f-score for each category
f1 = f1_score(y_testArray,pred, average=None)

print (f1)
print ("\n")
print (np.sum(f1)/3)

[0.75 0.8  0.88]


0.8099999999999999


In [167]:
# calculate f-score for all 
print ("F-score\n-----")
print ("Weighted: ", f1_score(y_testArray,pred, average='weighted'))
print ("Micro: ", f1_score(y_testArray,pred, average='micro'))
print ("Macro: ", f1_score(y_testArray,pred, average='macro'))

F-score
-----
Weighted:  0.8037254901960783
Micro:  0.8039215686274509
Macro:  0.8099999999999999


In [168]:
target_names = ['Homicide', 'Parricide', 'Murder']
print(classification_report(y_testArray, pred, target_names=target_names))

             precision    recall  f1-score   support

   Homicide       0.80      0.71      0.75        17
  Parricide       0.75      0.86      0.80        21
     Murder       0.92      0.85      0.88        13

avg / total       0.81      0.80      0.80        51



## hidden data

In [169]:
words1 = "Burlington 12-year-old Accused of killing two parents and wounding two siblings. Charged as a juvenile.".lower()
toks = w2v_tokenize_text(words1)
word_average = word_averaging(wv,toks)
clf.predict([word_average])

array(['parricide'], dtype='<U9')